In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

df =  pd.read_csv('winequality-red.csv',delimiter=';')
df.dropna(inplace=True)

df['Y'] = np.where(df['Y'] >= 6, 1, 0)

In [2]:
#df = df.sample(n=1000, random_state=42)

In [3]:
import pandas as pd
import numpy as np

Y_column = df['Y'].copy()
df.drop('Y', axis=1, inplace=True)

# Identify categorical data (change this based on your actual data)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Standardize only the continuous (non-categorical) columns
continuous_cols = df.columns.difference(categorical_cols)  # Gets the difference, i.e., continuous cols
df[continuous_cols] = (df[continuous_cols] - df[continuous_cols].mean()) / df[continuous_cols].std()

# Filter out outliers in continuous data (|z-score| > 5)
mask = (np.abs(df[continuous_cols]) < 5).all(axis=1)
df = df[mask]

# Reattach the target variable 'Y' to the DataFrame
df['Y'] = Y_column[mask]

In [4]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()

# Apply Label Encoding to each categorical column
for column in categorical_cols:
    # Ensure the column is of type object (string) or category
    if df[column].dtype == 'object' or df[column].dtype.name == 'category':
        df[column] = le.fit_transform(df[column])
        
df['Y'], unique = pd.factorize(df['Y'])

In [5]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dictionary of models and their reduced hyperparameter grids
models = {
    'Logistic Regression': (LogisticRegression(max_iter=1000), {
        'C': [0.01, 0.1, 1],
        'solver': ['liblinear', 'lbfgs']
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }),
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10],
        'min_samples_split': [2, 5]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 4]
    }),
    'XGBoost': (XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 4]
    }),
    'LightGBM': (LGBMClassifier(), {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'num_leaves': [31, 50]
    }),
    'CatBoost': (CatBoostClassifier(verbose=0), {
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.1],
        'depth': [4, 6]
    })
}

# Dictionary to store results
results = {}

for name, (model, param_grid) in models.items():
    start_time = time.time()
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_

    # Measure training time for best parameters
    best_param_train_start = time.time()
    best_model.fit(X_train_scaled, y_train)
    best_param_train_time = time.time() - best_param_train_start

    # Measure inference time for best parameters
    inference_start_time = time.time()
    y_pred = best_model.predict(X_test_scaled)
    inference_time = time.time() - inference_start_time
    
    # Calculate total computation time
    computation_time = time.time() - start_time
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    if len(np.unique(y)) == 2:  # Binary classification
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])
    else:  # Multiclass classification
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr', average='macro')

    results[name] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'Training Time (Best Params)': best_param_train_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': grid_search.best_params_
    }

# Convert results to a DataFrame
result = pd.DataFrame(results).T

print(result)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:15:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:15:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 361, number of negative: 415
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 847
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465206 -> initscore=-0.139401
[LightGBM] [Info] Start training from score -0.139401
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neural_network import MLPClassifier

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model and its hyperparameter grid
mlp = MLPClassifier(max_iter=1000, random_state=42)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Perform GridSearchCV with StratifiedKFold
start_time = time.time()
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Measure training time for best parameters
best_param_train_start = time.time()
best_model.fit(X_train_scaled, y_train)
training_time = time.time() - best_param_train_start

# Measure inference time for best parameters
inference_start_time = time.time()
y_pred = best_model.predict(X_test_scaled)
inference_time = time.time() - inference_start_time

# Calculate total computation time
computation_time = time.time() - start_time

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
if len(np.unique(y)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])
else:  # Multiclass classification
    auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr', average='macro')

# Store results in the existing result DataFrame
result.loc['MLP'] = [accuracy, auc, training_time, inference_time, computation_time, grid_search.best_params_]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in grid_search.best_params_.items():
    print(f"{param}: {value}")

c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Regression                          0.0                 4.049861   
KNN                                     0.004881                 0.248299   
Decision Tree                                0.0          

c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [7]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Define the DNN model
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(DNN, self).__init__()
        layers = []
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            input_dim = hidden_dim
        layers.append(nn.Linear(input_dim, output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to tune
    hidden_dims = [trial.suggest_int(f'hidden_dim_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = DNN(input_dim, hidden_dims, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Start timing the entire process
start_time = time.time()

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = DNN(input_dim, [best_params[f'hidden_dim_{i}'] for i in range(3)], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['DNN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:19:59,336] A new study created in memory with name: no-name-7449ee2f-298a-4134-9e5c-edd4dc861a46


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1656234290.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:20:16,422] Trial 0 finished with value: 0.6717948717948717 and parameters: {'hidden_dim_0': 235, 'hidden_dim_1': 183, 'hidden_dim_2': 182, 'learning_rate': 0.039847770232168886, 'batch_size': 32, 'num_epochs': 66}. Best is trial 0 with value: 0.6717948717948717.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1656234290.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:20:16,803] Tria

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Regression                          0.0                 4.049861   
KNN                                     0.004881                

In [8]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class CrossLayer(nn.Module):
    def __init__(self, input_dim):
        super(CrossLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(input_dim, 1))
        self.bias = nn.Parameter(torch.Tensor(input_dim, 1))
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x0, x):
        x = x.unsqueeze(2)
        x0 = x0.unsqueeze(2)
        interaction = torch.matmul(x0, torch.matmul(x.transpose(1, 2), self.weight))
        return x0.squeeze(2) + interaction.squeeze(2) + self.bias.T

class DCN(nn.Module):
    def __init__(self, input_dim, cross_layers, hidden_layers, output_dim):
        super(DCN, self).__init__()
        self.cross_layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(cross_layers)])
        
        deep_layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            deep_layers.append(nn.ReLU())
        self.deep_net = nn.Sequential(*deep_layers)
        
        self.final_layer = nn.Linear(input_dim + hidden_layers[-1], output_dim)

    def forward(self, x):
        cross_out = x
        for layer in self.cross_layers:
            cross_out = layer(x, cross_out)
        deep_out = self.deep_net(x)
        concat_out = torch.cat([cross_out, deep_out], dim=1)
        return self.final_layer(concat_out)

def objective(trial):
    # Define hyperparameters to tune
    cross_layers = trial.suggest_int('cross_layers', 1, 5)
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = DCN(input_dim, cross_layers, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Start timing the entire process
start_time = time.time()

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = DCN(input_dim, best_params['cross_layers'], 
                 [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                 output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['DCN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:21:33,469] A new study created in memory with name: no-name-9b0175e2-7e6f-4a9b-9b07-0d9a97c7e8ab


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3493045964.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:21:36,380] Trial 0 finished with value: 0.6820512820512821 and parameters: {'cross_layers': 2, 'hidden_layer_0': 204, 'hidden_layer_1': 220, 'hidden_layer_2': 80, 'learning_rate': 0.0788376381887241, 'batch_size': 256, 'num_epochs': 45}. Best is trial 0 with value: 0.6820512820512821.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3493045964.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-0

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   
DCN                  0.723077  0.778793                    5.800046   

                    Inference Time (Best Params) Computation Time (Total)  \
Logistic Regression                          0.0                 4.049

In [9]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class WideAndDeepNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(WideAndDeepNetwork, self).__init__()
        
        # Wide part
        self.wide = nn.Linear(input_dim, output_dim)
        
        # Deep part
        deep_layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            deep_layers.append(nn.ReLU())
        deep_layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.deep = nn.Sequential(*deep_layers)

    def forward(self, x):
        wide_out = self.wide(x)
        deep_out = self.deep(x)
        return wide_out + deep_out

def objective(trial):
    # Define hyperparameters to tune
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = WideAndDeepNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Start timing the entire process
start_time = time.time()

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = WideAndDeepNetwork(input_dim, 
                                [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                                output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['Wide_and_Deep'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:23:41,009] A new study created in memory with name: no-name-71699cb6-bb7f-474c-9c91-096da33e2d23


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3403281423.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:23:51,254] Trial 0 finished with value: 0.7589743589743589 and parameters: {'hidden_layer_0': 134, 'hidden_layer_1': 83, 'hidden_layer_2': 109, 'learning_rate': 0.0007483649547044918, 'batch_size': 32, 'num_epochs': 83}. Best is trial 0 with value: 0.7589743589743589.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3403281423.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:23:52,307

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   
DCN                  0.723077  0.778793                    5.800046   
Wide_and_Deep        0.774359  0.835693                    2.907177   

                    Inference Time (Best Params) Computation Time (Total)  \

In [10]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    # Train XGBoost model
    xgb_model = XGBClassifier(**xgb_params, use_label_encoder=False, eval_metric='mlogloss')
    xgb_model.fit(X_train_scaled, y_train)
    
    # Extract features using XGBoost
    X_train_transformed = xgb_model.apply(X_train_scaled)
    X_test_transformed = xgb_model.apply(X_test_scaled)
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train.values).to(device)
    y_test_tensor = torch.LongTensor(y_test.values).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = []
    for i in range(3):  # Allow up to 3 hidden layers
        if trial.suggest_categorical(f'use_hidden_layer_{i}', [True, False]):
            hidden_layers.append(trial.suggest_int(f'hidden_layer_{i}', 32, 256))
    
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final XGBoost model with the best hyperparameters
xgb_best_params = {
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'learning_rate': best_params['xgb_learning_rate'],
    'subsample': best_params['subsample'],
    'colsample_bytree': best_params['colsample_bytree']
}
xgb_model = XGBClassifier(**xgb_best_params, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_scaled, y_train)

# Extract features using XGBoost
X_train_transformed = xgb_model.apply(X_train_scaled)
X_test_transformed = xgb_model.apply(X_test_scaled)

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}

# Dynamically determine the number of hidden layers
for i in range(3):  # Assuming max 3 hidden layers
    if f'use_hidden_layer_{i}' in best_params and best_params[f'use_hidden_layer_{i}']:
        nn_best_params['hidden_layers'].append(best_params[f'hidden_layer_{i}'])

input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Assuming 'result' DataFrame is defined elsewhere
result.loc['XGBoost + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['XGBoost + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:25:17,315] A new study created in memory with name: no-name-cc01aa6d-730a-48ca-b724-101c9adc575b


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2693607433.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:25:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2693607433.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   
DCN                  0.723077  0.778793                    5.800046   
Wide_and_Deep        0.774359  0.835693                    2.907177   
XGBoost + NN         0.697436    0.8209                    3.071482   

     

In [11]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from lightgbm import LGBMClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for LightGBM
    lgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    # Train LightGBM model
    lgb_model = LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train_scaled, y_train)
    
    # Extract features using LightGBM
    X_train_transformed = lgb_model.predict_proba(X_train_scaled)
    X_test_transformed = lgb_model.predict_proba(X_test_scaled)
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train.values).to(device)
    y_test_tensor = torch.LongTensor(y_test.values).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final LightGBM model with the best hyperparameters
lgb_best_params = {
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'learning_rate': best_params['lgb_learning_rate'],
    'num_leaves': best_params['num_leaves'],
    'subsample': best_params['subsample'],
    'colsample_bytree': best_params['colsample_bytree']
}
lgb_model = LGBMClassifier(**lgb_best_params)
lgb_model.fit(X_train_scaled, y_train)

# Extract features using LightGBM
X_train_transformed = lgb_model.predict_proba(X_train_scaled)
X_test_transformed = lgb_model.predict_proba(X_test_scaled)

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['LightGBM + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:27:04,788] A new study created in memory with name: no-name-9aafeffd-ea8f-43ea-8934-dd4af9c60c7b


Using device: cpu
[LightGBM] [Info] Number of positive: 362, number of negative: 414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.466495 -> initscore=-0.134222
[LightGBM] [Info] Start training from score -0.134222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3458157511.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3458157511.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:27:11,915] Trial 0 finished with value: 0.6820512820512821 and parameters: {'n_estimators': 171, 'max_depth': 6, 'lgb_learning_rate': 0.0007303950843652628, 'num_leaves': 91, 'subsample': 0.5890687346881405, 'colsample_bytree': 0.8765431388466591, 'hidden_layer_0': 149, 'hidden_laye

[LightGBM] [Info] Number of positive: 362, number of negative: 414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.466495 -> initscore=-0.134222
[LightGBM] [Info] Start training from score -0.134222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2024-07-26 21:27:17,370] Trial 1 finished with value: 0.676923076923077 and parameters: {'n_estimators': 155, 'max_depth': 5, 'lgb_learning_rate': 0.0008708741477612304, 'num_leaves': 72, 'subsample': 0.668279713210599, 'colsample_bytree': 0.9867303864548873, 'hidden_layer_0': 175, 'hidden_layer_1': 211, 'hidden_layer_2': 108, 'nn_learning_rate': 0.002780997393517395, 'batch_size': 128, 'num_epochs': 89}. Best is trial 0 with value: 0.6820512820512821.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3458157511.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3458157511.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github

...

[LightGBM] [Info] Number of positive: 362, number of negative: 414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.466495 -> initscore=-0.134222
[LightGBM] [Info] Start training from score -0.134222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2024-07-26 21:28:40,552] Trial 18 finished with value: 0.7538461538461538 and parameters: {'n_estimators': 272, 'max_depth': 5, 'lgb_learning_rate': 0.04308524863136817, 'num_leaves': 50, 'subsample': 0.5239516435239182, 'colsample_bytree': 0.7838011324150067, 'hidden_layer_0': 119, 'hidden_layer_1': 119, 'hidden_layer_2': 154, 'nn_learning_rate': 0.0002648370320288155, 'batch_size': 32, 'num_epochs': 39}. Best is trial 15 with value: 0.7743589743589744.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3458157511.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3458157511.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gith

[LightGBM] [Info] Number of positive: 362, number of negative: 414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000124 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.466495 -> initscore=-0.134222
[LightGBM] [Info] Start training from score -0.134222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2024-07-26 21:28:43,605] Trial 19 finished with value: 0.7435897435897436 and parameters: {'n_estimators': 293, 'max_depth': 5, 'lgb_learning_rate': 0.012423604624350425, 'num_leaves': 100, 'subsample': 0.7010085370253263, 'colsample_bytree': 0.646119129265326, 'hidden_layer_0': 69, 'hidden_layer_1': 171, 'hidden_layer_2': 197, 'nn_learning_rate': 0.0015715305325937172, 'batch_size': 256, 'num_epochs': 74}. Best is trial 15 with value: 0.7743589743589744.


[LightGBM] [Info] Number of positive: 362, number of negative: 414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 856
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.466495 -> initscore=-0.134222
[LightGBM] [Info] Start training from score -0.134222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [12]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from catboost import CatBoostClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for CatBoost
    catboost_params = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('catboost_learning_rate', 1e-4, 1e-1),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-4, 1e-1),
        'border_count': trial.suggest_int('border_count', 32, 255)
    }

    # Train CatBoost model
    catboost_model = CatBoostClassifier(**catboost_params, verbose=0)
    catboost_model.fit(X_train_scaled, y_train)
    
    # Extract features using CatBoost
    X_train_transformed = catboost_model.predict_proba(X_train_scaled)
    X_test_transformed = catboost_model.predict_proba(X_test_scaled)
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train.values).to(device)
    y_test_tensor = torch.LongTensor(y_test.values).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final CatBoost model with the best hyperparameters
catboost_best_params = {
    'iterations': best_params['iterations'],
    'depth': best_params['depth'],
    'learning_rate': best_params['catboost_learning_rate'],
    'l2_leaf_reg': best_params['l2_leaf_reg'],
    'border_count': best_params['border_count']
}
catboost_model = CatBoostClassifier(**catboost_best_params, verbose=0)
catboost_model.fit(X_train_scaled, y_train)

# Extract features using CatBoost
X_train_transformed = catboost_model.predict_proba(X_train_scaled)
X_test_transformed = catboost_model.predict_proba(X_test_scaled)

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['CatBoost + NN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:28:49,038] A new study created in memory with name: no-name-172a0f46-1890-408d-a37b-c702268bd013


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1954938432.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('catboost_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1954938432.py:55: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1954938432.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.sugg

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   
DCN                  0.723077  0.778793                    5.800046   
Wide_and_Deep        0.774359  0.835693                    2.907177   
XGBoost + NN         0.697436    0.8209                    3.071482   
LightG

In [13]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class AutoInt(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
        super(AutoInt, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)
        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(embedding_dim, num_heads) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(embedding_dim, 1)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        for attn_layer in self.attention_layers:
            x, _ = attn_layer(x, x, x)
        x = x.squeeze(1)
        x = self.fc(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for AutoInt
    num_heads = trial.suggest_int('num_heads', 1, 8)
    embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    
    # Train AutoInt model
    autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    for epoch in range(10):  # Fixed number of epochs for AutoInt
        autoint_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = autoint_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Extract features using AutoInt
    autoint_model.eval()
    with torch.no_grad():
        X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final AutoInt model with the best hyperparameters
embedding_dim = best_params['embedding_dim']
num_heads = best_params['num_heads']
num_layers = best_params['num_layers']
autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in range(10):  # Fixed number of epochs for AutoInt
    autoint_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = autoint_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Extract features using AutoInt
autoint_model.eval()
with torch.no_grad():
    X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
    X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['AutoInt'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:30:48,199] A new study created in memory with name: no-name-9eccdf7f-7363-40f3-98b7-3f01ae87034a


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\652687641.py:106: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:30:56,223] Trial 0 finished with value: 0.7794871794871795 and parameters: {'num_heads': 8, 'embedding_dim': 56, 'num_layers': 3, 'hidden_layer_0': 138, 'hidden_layer_1': 155, 'hidden_layer_2': 68, 'nn_learning_rate': 0.0008061114172516176, 'batch_size': 64, 'num_epochs': 61}. Best is trial 0 with value: 0.7794871794871795.
c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [3, 64] and step=3, but the range is not divisible by `step`. It will be replaced by [3, 63].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\652687641.py:106: FutureWarni

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   
DCN                  0.723077  0.778793                    5.800046   
Wide_and_Deep        0.774359  0.835693                    2.907177   
XGBoost + NN         0.697436    0.8209                    3.071482   
LightG

In [14]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class FTTransformer(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
        super(FTTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(embedding_dim, input_dim)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x)
        x = x.squeeze(1)
        x = self.fc(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for FT-Transformer
    num_heads = trial.suggest_int('num_heads', 1, 8)
    embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    
    # Train FT-Transformer model
    ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    for epoch in range(10):  # Fixed number of epochs for FT-Transformer
        ft_transformer_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = ft_transformer_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Extract features using FT-Transformer
    ft_transformer_model.eval()
    with torch.no_grad():
        X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()
    
    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    
    # Define hyperparameters to tune for Neural Network
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final FT-Transformer model with the best hyperparameters
embedding_dim = best_params['embedding_dim']
num_heads = best_params['num_heads']
num_layers = best_params['num_layers']
ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in range(10):  # Fixed number of epochs for FT-Transformer
    ft_transformer_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = ft_transformer_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Extract features using FT-Transformer
ft_transformer_model.eval()
with torch.no_grad():
    X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
    X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()

# Convert to PyTorch tensors
X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

# Train the final Neural Network model with the best hyperparameters
nn_best_params = {
    'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
    'learning_rate': best_params['nn_learning_rate'],
    'batch_size': best_params['batch_size'],
    'num_epochs': best_params['num_epochs']
}
input_dim = X_train_transformed.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(nn_best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_transformed_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['FT-Transformer'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:33:22,387] A new study created in memory with name: no-name-da008a59-395d-4d19-8668-ae5a3353f64b


Using device: cpu


c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 11])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 11])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2501686716.py:106: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nn_learning_rate = tria

                     Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression  0.748718  0.796644                    0.003991   
KNN                  0.753846  0.840861                    0.001994   
Decision Tree        0.728205  0.749789                    0.004987   
Random Forest             0.8  0.864816                    0.727075   
Gradient Boosting         0.8  0.861439                    0.513596   
XGBoost              0.764103  0.853525                      0.1127   
LightGBM             0.789744  0.866083                    0.057846   
CatBoost             0.753846  0.843288                    0.245005   
MLP                  0.764103  0.838863                    3.545068   
DNN                  0.774359  0.836116                    2.187616   
DCN                  0.723077  0.778793                    5.800046   
Wide_and_Deep        0.774359  0.835693                    2.907177   
XGBoost + NN         0.697436    0.8209                    3.071482   
LightG

In [15]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            if i == 0:
                layers.append(nn.Linear(input_dim, hidden_layers[i]))
            else:
                layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for Neural Network
    num_layers = trial.suggest_int('num_layers', 1, 5)
    hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(num_layers)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Neural Network model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final Neural Network model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = NeuralNetwork(input_dim, 
                           [best_params[f'hidden_layer_{i}'] for i in range(best_params['num_layers'])], 
                           output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_pred = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        auc = roc_auc_score(y_true, proba)
    else:  # Multi-class classification
        proba = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['Neural Architecture Search'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:37:52,750] A new study created in memory with name: no-name-ecd9127e-b104-49a2-86c9-6f016ad41ae4


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3158987854.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:37:54,045] Trial 0 finished with value: 0.6974358974358974 and parameters: {'num_layers': 5, 'hidden_layer_0': 256, 'hidden_layer_1': 195, 'hidden_layer_2': 65, 'hidden_layer_3': 63, 'hidden_layer_4': 35, 'learning_rate': 0.027251445591806936, 'batch_size': 256, 'num_epochs': 30}. Best is trial 0 with value: 0.6974358974358974.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\3158987854.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logun

                            Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression         0.748718  0.796644                    0.003991   
KNN                         0.753846  0.840861                    0.001994   
Decision Tree               0.728205  0.749789                    0.004987   
Random Forest                    0.8  0.864816                    0.727075   
Gradient Boosting                0.8  0.861439                    0.513596   
XGBoost                     0.764103  0.853525                      0.1127   
LightGBM                    0.789744  0.866083                    0.057846   
CatBoost                    0.753846  0.843288                    0.245005   
MLP                         0.764103  0.838863                    3.545068   
DNN                         0.774359  0.836116                    2.187616   
DCN                         0.723077  0.778793                    5.800046   
Wide_and_Deep               0.774359  0.835693                  

In [16]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class NODE(nn.Module):
    def __init__(self, input_dim, num_layers, num_trees, tree_dim, output_dim):
        super(NODE, self).__init__()
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            layer = nn.ModuleList()
            for _ in range(num_trees):
                tree = nn.Sequential(
                    nn.Linear(input_dim, tree_dim),
                    nn.ReLU(),
                    nn.Linear(tree_dim, 1)
                )
                layer.append(tree)
            self.layers.append(layer)
        self.output = nn.Linear(num_layers * num_trees, output_dim)

    def forward(self, x):
        tree_outputs = []
        for layer in self.layers:
            layer_outputs = []
            for tree in layer:
                layer_outputs.append(tree(x))
            layer_output = torch.cat(layer_outputs, dim=1)
            tree_outputs.append(layer_output)
        x = torch.cat(tree_outputs, dim=1)
        return self.output(x)

def objective(trial):
    # Define hyperparameters to tune for NODE
    num_layers = trial.suggest_int('num_layers', 1, 5)
    num_trees = trial.suggest_int('num_trees', 1, 10)
    tree_dim = trial.suggest_int('tree_dim', 8, 64)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the NODE model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = NODE(input_dim, num_layers, num_trees, tree_dim, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final NODE model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = NODE(input_dim, 
                  best_params['num_layers'], 
                  best_params['num_trees'], 
                  best_params['tree_dim'], 
                  output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.cpu().numpy()
    proba = torch.softmax(outputs, dim=1).cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        auc = roc_auc_score(y_true, proba[:, 1])
    else:  # Multi-class classification
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['NODE'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:39:35,442] A new study created in memory with name: no-name-41b9623f-8eba-40ce-8cca-f15a80fda323


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2066003851.py:70: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:39:56,396] Trial 0 finished with value: 0.7487179487179487 and parameters: {'num_layers': 4, 'num_trees': 8, 'tree_dim': 23, 'learning_rate': 0.051167047104067796, 'batch_size': 128, 'num_epochs': 90}. Best is trial 0 with value: 0.7487179487179487.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2066003851.py:70: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:40:28,571] Trial 1 finished 

                            Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression         0.748718  0.796644                    0.003991   
KNN                         0.753846  0.840861                    0.001994   
Decision Tree               0.728205  0.749789                    0.004987   
Random Forest                    0.8  0.864816                    0.727075   
Gradient Boosting                0.8  0.861439                    0.513596   
XGBoost                     0.764103  0.853525                      0.1127   
LightGBM                    0.789744  0.866083                    0.057846   
CatBoost                    0.753846  0.843288                    0.245005   
MLP                         0.764103  0.838863                    3.545068   
DNN                         0.774359  0.836116                    2.187616   
DCN                         0.723077  0.778793                    5.800046   
Wide_and_Deep               0.774359  0.835693                  

In [17]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

def objective(trial):
    # Define hyperparameters to tune for TabNet
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the TabNet model
    model = TabNetClassifier(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        device_name=device
    )

    # Training
    model.fit(
        X_train=X_train_scaled, y_train=y_train.values,
        eval_set=[(X_test_scaled, y_test.values)],
        eval_name=['val'],
        eval_metric=['accuracy'],
        max_epochs=num_epochs,
        patience=10,
        batch_size=batch_size,
        virtual_batch_size=batch_size // 2,
        num_workers=0,
        drop_last=False
    )

    # Evaluation
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final TabNet model with the best hyperparameters
best_model = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    device_name=device
)

training_start_time = time.time()
best_model.fit(
    X_train=X_train_scaled, y_train=y_train.values,
    eval_set=[(X_test_scaled, y_test.values)],
    eval_name=['val'],
    eval_metric=['accuracy'],
    max_epochs=best_params['num_epochs'],
    patience=10,
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['batch_size'] // 2,
    num_workers=0,
    drop_last=False
)
training_time = time.time() - training_start_time

# Evaluation
y_pred = best_model.predict(X_test_scaled)
inference_start_time = time.time()
y_pred_proba = best_model.predict_proba(X_test_scaled)
inference_time = time.time() - inference_start_time

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
if len(np.unique(y)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
else:  # Multiclass classification
    auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['TabNet'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:44:31,670] A new study created in memory with name: no-name-8ee6defe-9ae0-49e3-84b8-860d75358470


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1744437722.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1744437722.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.3455  | val_accuracy: 0.53846 |  0:00:02s
epoch 1  | loss: 1.23687 | val_accuracy: 0.54872 |  0:00:04s
epoch 2  | loss: 1.10571 | val_accuracy: 0.6     |  0:00:06s
epoch 3  | loss: 1.05679 | val_accuracy: 0.56923 |  0:00:09s
epoch 4  | loss: 1.11113 | val_accuracy: 0.58974 |  0:00:11s
epoch 5  | loss: 1.04627 | val_accuracy: 0.61538 |  0:00:13s
epoch 6  | loss: 0.92673 | val_accuracy: 0.57949 |  0:00:15s
epoch 7  | loss: 0.8847  | val_accuracy: 0.58974 |  0:00:18s
epoch 8  | loss: 0.91821 | val_accuracy: 0.62564 |  0:00:20s
epoch 9  | loss: 0.9308  | val_accuracy: 0.58974 |  0:00:22s
epoch 10 | loss: 0.87777 | val_accuracy: 0.59487 |  0:00:24s
epoch 11 | loss: 0.8088  | val_accuracy: 0.61026 |  0:00:27s
epoch 12 | loss: 0.84667 | val_accuracy: 0.61538 |  0:00:29s
epoch 13 | loss: 0.80574 | val_accuracy: 0.61026 |  0:00:31s
epoch 14 | loss: 0.82285 | val_accuracy: 0.61538 |  0:00:33s
epoch 15 | loss: 0.78058 | val_accuracy: 0.64103 |  0:00:36s
epoch 16 | loss: 0.82244

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 21:45:32,038] Trial 0 finished with value: 0.6410256410256411 and parameters: {'n_d': 59, 'n_a': 33, 'n_steps': 6, 'gamma': 1.5476170306282893, 'lambda_sparse': 8.999343538588935e-06, 'learning_rate': 0.00039879252081584095, 'batch_size': 32, 'num_epochs': 75}. Best is trial 0 with value: 0.6410256410256411.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1744437722.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\1744437722.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v

...

epoch 0  | loss: 1.06468 | val_accuracy: 0.6359  |  0:00:01s
epoch 1  | loss: 0.70026 | val_accuracy: 0.68205 |  0:00:03s
epoch 2  | loss: 0.72526 | val_accuracy: 0.67692 |  0:00:05s
epoch 3  | loss: 0.67881 | val_accuracy: 0.69231 |  0:00:07s
epoch 4  | loss: 0.6566  | val_accuracy: 0.65128 |  0:00:09s
epoch 5  | loss: 0.64935 | val_accuracy: 0.67692 |  0:00:11s
epoch 6  | loss: 0.58683 | val_accuracy: 0.65641 |  0:00:12s
epoch 7  | loss: 0.58752 | val_accuracy: 0.68205 |  0:00:16s
epoch 8  | loss: 0.61817 | val_accuracy: 0.69231 |  0:00:17s
epoch 9  | loss: 0.60115 | val_accuracy: 0.70769 |  0:00:20s
epoch 10 | loss: 0.57936 | val_accuracy: 0.68718 |  0:00:21s
epoch 11 | loss: 0.58927 | val_accuracy: 0.68718 |  0:00:23s
epoch 12 | loss: 0.558   | val_accuracy: 0.70769 |  0:00:25s
epoch 13 | loss: 0.56991 | val_accuracy: 0.70769 |  0:00:28s
epoch 14 | loss: 0.56246 | val_accuracy: 0.70769 |  0:00:30s
epoch 15 | loss: 0.54717 | val_accuracy: 0.69231 |  0:00:32s
epoch 16 | loss: 0.5657 

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 21:53:41,798] Trial 19 finished with value: 0.7794871794871795 and parameters: {'n_d': 33, 'n_a': 8, 'n_steps': 5, 'gamma': 1.108918887339927, 'lambda_sparse': 6.34631345516932e-05, 'learning_rate': 0.0034942897183386835, 'batch_size': 64, 'num_epochs': 63}. Best is trial 19 with value: 0.7794871794871795.
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.06468 | val_accuracy: 0.6359  |  0:00:00s
epoch 1  | loss: 0.70026 | val_accuracy: 0.68205 |  0:00:01s
epoch 2  | loss: 0.72526 | val_accuracy: 0.67692 |  0:00:02s
epoch 3  | loss: 0.67881 | val_accuracy: 0.69231 |  0:00:03s
epoch 4  | loss: 0.6566  | val_accuracy: 0.65128 |  0:00:04s
epoch 5  | loss: 0.64935 | val_accuracy: 0.67692 |  0:00:05s
epoch 6  | loss: 0.58683 | val_accuracy: 0.65641 |  0:00:05s
epoch 7  | loss: 0.58752 | val_accuracy: 0.68205 |  0:00:06s
epoch 8  | loss: 0.61817 | val_accuracy: 0.69231 |  0:00:07s
epoch 9  | loss: 0.60115 | val_accuracy: 0.70769 |  0:00:08s
epoch 10 | loss: 0.57936 | val_accuracy: 0.68718 |  0:00:09s
epoch 11 | loss: 0.58927 | val_accuracy: 0.68718 |  0:00:09s
epoch 12 | loss: 0.558   | val_accuracy: 0.70769 |  0:00:10s
epoch 13 | loss: 0.56991 | val_accuracy: 0.70769 |  0:00:11s
epoch 14 | loss: 0.56246 | val_accuracy: 0.70769 |  0:00:12s
epoch 15 | loss: 0.54717 | val_accuracy: 0.69231 |  0:00:13s
epoch 16 | loss: 0.5657 

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


                            Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression         0.748718  0.796644                    0.003991   
KNN                         0.753846  0.840861                    0.001994   
Decision Tree               0.728205  0.749789                    0.004987   
Random Forest                    0.8  0.864816                    0.727075   
Gradient Boosting                0.8  0.861439                    0.513596   
XGBoost                     0.764103  0.853525                      0.1127   
LightGBM                    0.789744  0.866083                    0.057846   
CatBoost                    0.753846  0.843288                    0.245005   
MLP                         0.764103  0.838863                    3.545068   
DNN                         0.774359  0.836116                    2.187616   
DCN                         0.723077  0.778793                    5.800046   
Wide_and_Deep               0.774359  0.835693                  

In [18]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class KAN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KAN, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h = self.activation(self.hidden_layer(x))
        out = self.output_layer(h)
        return out

def objective(trial):
    # Define hyperparameters to tune for KAN
    hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the KAN model
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    model = KAN(input_dim, hidden_dim, output_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final KAN model with the best hyperparameters
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
best_model = KAN(input_dim, 
                 best_params['hidden_dim'], 
                 output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.cpu().numpy()
    proba = torch.softmax(outputs, dim=1).cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    
    if output_dim == 2:  # Binary classification
        auc = roc_auc_score(y_true, proba[:, 1])
    else:  # Multi-class classification
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['KAN'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:54:38,885] A new study created in memory with name: no-name-d6b10731-f73c-45c5-ad13-a68318d79bfe


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2557614817.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:54:41,019] Trial 0 finished with value: 0.7230769230769231 and parameters: {'hidden_dim': 186, 'learning_rate': 0.0001924859163305034, 'batch_size': 128, 'num_epochs': 54}. Best is trial 0 with value: 0.7230769230769231.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\2557614817.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 21:54:41,940] Trial 1 finished with value: 0.728205128205128

                            Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression         0.748718  0.796644                    0.003991   
KNN                         0.753846  0.840861                    0.001994   
Decision Tree               0.728205  0.749789                    0.004987   
Random Forest                    0.8  0.864816                    0.727075   
Gradient Boosting                0.8  0.861439                    0.513596   
XGBoost                     0.764103  0.853525                      0.1127   
LightGBM                    0.789744  0.866083                    0.057846   
CatBoost                    0.753846  0.843288                    0.245005   
MLP                         0.764103  0.838863                    3.545068   
DNN                         0.774359  0.836116                    2.187616   
DCN                         0.723077  0.778793                    5.800046   
Wide_and_Deep               0.774359  0.835693                  

In [19]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

class SAINT(nn.Module):
    def __init__(self, input_dim, num_classes, dim, depth, heads, mlp_dim, dropout=0.1):
        super(SAINT, self).__init__()
        self.embeds = nn.Linear(input_dim, dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout),
            num_layers=depth
        )
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        x = self.embeds(x)
        x = x.unsqueeze(1)  # Add sequence dimension
        x = self.transformer(x)
        x = x.squeeze(1)  # Remove sequence dimension
        return self.mlp_head(x)

def objective(trial):
    # Define hyperparameters to tune for SAINT
    heads = trial.suggest_int('heads', 1, 8)
    dim = trial.suggest_int('dim', heads, 256, step=heads)  # Ensure dim is divisible by heads
    depth = trial.suggest_int('depth', 1, 6)
    mlp_dim = trial.suggest_int('mlp_dim', 32, 256)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the SAINT model
    input_dim = X_train.shape[1]
    num_classes = len(np.unique(y))
    model = SAINT(input_dim, num_classes, dim, depth, heads, mlp_dim, dropout).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final SAINT model with the best hyperparameters
input_dim = X_train.shape[1]
num_classes = len(np.unique(y))
best_model = SAINT(input_dim, num_classes, 
                   best_params['dim'], 
                   best_params['depth'], 
                   best_params['heads'], 
                   best_params['mlp_dim'], 
                   best_params['dropout']).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    outputs = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    
    _, predicted = torch.max(outputs, 1)
    y_pred = predicted.cpu().numpy()
    proba = torch.softmax(outputs, dim=1).cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    
    if num_classes == 2:  # Binary classification
        auc = roc_auc_score(y_true, proba[:, 1])
    else:  # Multi-class classification
        auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['SAINT'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 21:55:23,406] A new study created in memory with name: no-name-ef99e6f9-4a27-4d29-b141-0008901e9a55


Using device: cpu


c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [7, 256] and step=7, but the range is not divisible by `step`. It will be replaced by [7, 252].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\168735295.py:65: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-07-26 21:55:28,225] Trial 0 f

                            Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression         0.748718  0.796644                    0.003991   
KNN                         0.753846  0.840861                    0.001994   
Decision Tree               0.728205  0.749789                    0.004987   
Random Forest                    0.8  0.864816                    0.727075   
Gradient Boosting                0.8  0.861439                    0.513596   
XGBoost                     0.764103  0.853525                      0.1127   
LightGBM                    0.789744  0.866083                    0.057846   
CatBoost                    0.753846  0.843288                    0.245005   
MLP                         0.764103  0.838863                    3.545068   
DNN                         0.774359  0.836116                    2.187616   
DCN                         0.723077  0.778793                    5.800046   
Wide_and_Deep               0.774359  0.835693                  

In [20]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is already defined
X = df.drop('Y', axis=1)
y = df['Y']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

def objective(trial):
    # Define hyperparameters to tune for VIME-like model (using TabNet as proxy)
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the TabNet model
    model = TabNetClassifier(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        device_name=device
    )

    # Training
    model.fit(
        X_train=X_train_scaled, y_train=y_train.values,
        eval_set=[(X_test_scaled, y_test.values)],
        eval_name=['val'],
        eval_metric=['accuracy'],
        max_epochs=num_epochs,
        patience=10,
        batch_size=batch_size,
        virtual_batch_size=batch_size // 2,
        num_workers=0,
        drop_last=False
    )

    # Evaluation
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final TabNet model with the best hyperparameters
best_model = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    device_name=device
)

training_start_time = time.time()
best_model.fit(
    X_train=X_train_scaled, y_train=y_train.values,
    eval_set=[(X_test_scaled, y_test.values)],
    eval_name=['val'],
    eval_metric=['accuracy'],
    max_epochs=best_params['num_epochs'],
    patience=10,
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['batch_size'] // 2,
    num_workers=0,
    drop_last=False
)
training_time = time.time() - training_start_time

# Evaluation
y_pred = best_model.predict(X_test_scaled)
inference_start_time = time.time()
y_pred_proba = best_model.predict_proba(X_test_scaled)
inference_time = time.time() - inference_start_time

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
if len(np.unique(y)) == 2:  # Binary classification
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
else:  # Multiclass classification
    auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['VIME'] = [accuracy, auc, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 22:06:02,049] A new study created in memory with name: no-name-4e434e35-c773-4e31-bf25-2bb6ea7b287c


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\976418090.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\976418090.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.88328 | val_accuracy: 0.51795 |  0:00:01s
epoch 1  | loss: 1.15528 | val_accuracy: 0.6     |  0:00:01s
epoch 2  | loss: 1.03478 | val_accuracy: 0.5641  |  0:00:03s
epoch 3  | loss: 0.94845 | val_accuracy: 0.60513 |  0:00:04s
epoch 4  | loss: 0.82715 | val_accuracy: 0.61538 |  0:00:05s
epoch 5  | loss: 0.62127 | val_accuracy: 0.62564 |  0:00:05s
epoch 6  | loss: 0.65437 | val_accuracy: 0.60513 |  0:00:06s
epoch 7  | loss: 0.64205 | val_accuracy: 0.68718 |  0:00:07s
epoch 8  | loss: 0.67313 | val_accuracy: 0.59487 |  0:00:08s
epoch 9  | loss: 0.70581 | val_accuracy: 0.64615 |  0:00:09s
epoch 10 | loss: 0.74422 | val_accuracy: 0.64615 |  0:00:10s
epoch 11 | loss: 0.60619 | val_accuracy: 0.66667 |  0:00:11s
epoch 12 | loss: 0.58805 | val_accuracy: 0.61538 |  0:00:13s
epoch 13 | loss: 0.58355 | val_accuracy: 0.65128 |  0:00:14s
epoch 14 | loss: 0.57404 | val_accuracy: 0.66154 |  0:00:15s
epoch 15 | loss: 0.58828 | val_accuracy: 0.68205 |  0:00:16s
epoch 16 | loss: 0.60357

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 22:06:21,717] Trial 0 finished with value: 0.6871794871794872 and parameters: {'n_d': 25, 'n_a': 55, 'n_steps': 9, 'gamma': 1.9612932175244775, 'lambda_sparse': 1.5699959648691783e-05, 'learning_rate': 0.03553586705153356, 'batch_size': 128, 'num_epochs': 34}. Best is trial 0 with value: 0.6871794871794872.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\976418090.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6536\976418090.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0

...

epoch 0  | loss: 1.32713 | val_accuracy: 0.63077 |  0:00:01s
epoch 1  | loss: 0.69975 | val_accuracy: 0.61026 |  0:00:02s
epoch 2  | loss: 0.63773 | val_accuracy: 0.60513 |  0:00:03s
epoch 3  | loss: 0.69081 | val_accuracy: 0.64615 |  0:00:04s
epoch 4  | loss: 0.58557 | val_accuracy: 0.67692 |  0:00:05s
epoch 5  | loss: 0.60118 | val_accuracy: 0.67179 |  0:00:06s
epoch 6  | loss: 0.6362  | val_accuracy: 0.61538 |  0:00:07s
epoch 7  | loss: 0.63918 | val_accuracy: 0.70256 |  0:00:08s
epoch 8  | loss: 0.57456 | val_accuracy: 0.71282 |  0:00:10s
epoch 9  | loss: 0.59137 | val_accuracy: 0.68718 |  0:00:11s
epoch 10 | loss: 0.55959 | val_accuracy: 0.72308 |  0:00:12s
epoch 11 | loss: 0.55957 | val_accuracy: 0.70256 |  0:00:13s
epoch 12 | loss: 0.549   | val_accuracy: 0.66154 |  0:00:15s
epoch 13 | loss: 0.58846 | val_accuracy: 0.67179 |  0:00:16s
epoch 14 | loss: 0.57942 | val_accuracy: 0.72308 |  0:00:17s
epoch 15 | loss: 0.55671 | val_accuracy: 0.68718 |  0:00:18s
epoch 16 | loss: 0.57221

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 22:17:33,676] Trial 19 finished with value: 0.7230769230769231 and parameters: {'n_d': 16, 'n_a': 8, 'n_steps': 9, 'gamma': 1.0609717646774035, 'lambda_sparse': 4.068106497222082e-05, 'learning_rate': 0.041600966352928884, 'batch_size': 64, 'num_epochs': 74}. Best is trial 10 with value: 0.7692307692307693.
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.86321 | val_accuracy: 0.61538 |  0:00:01s
epoch 1  | loss: 1.01606 | val_accuracy: 0.6     |  0:00:03s
epoch 2  | loss: 0.85721 | val_accuracy: 0.61538 |  0:00:05s
epoch 3  | loss: 0.75975 | val_accuracy: 0.66667 |  0:00:07s
epoch 4  | loss: 0.81256 | val_accuracy: 0.62564 |  0:00:09s
epoch 5  | loss: 0.62794 | val_accuracy: 0.65128 |  0:00:11s
epoch 6  | loss: 0.6372  | val_accuracy: 0.68205 |  0:00:14s
epoch 7  | loss: 0.58164 | val_accuracy: 0.63077 |  0:00:17s
epoch 8  | loss: 0.59999 | val_accuracy: 0.69744 |  0:00:19s
epoch 9  | loss: 0.53638 | val_accuracy: 0.67179 |  0:00:20s
epoch 10 | loss: 0.55047 | val_accuracy: 0.65641 |  0:00:22s
epoch 11 | loss: 0.53529 | val_accuracy: 0.68205 |  0:00:24s
epoch 12 | loss: 0.55651 | val_accuracy: 0.69744 |  0:00:25s
epoch 13 | loss: 0.51592 | val_accuracy: 0.70256 |  0:00:27s
epoch 14 | loss: 0.54308 | val_accuracy: 0.76923 |  0:00:29s
epoch 15 | loss: 0.5295  | val_accuracy: 0.71795 |  0:00:32s
epoch 16 | loss: 0.56023

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


                            Accuracy AUC Score Training Time (Best Params)  \
Logistic Regression         0.748718  0.796644                    0.003991   
KNN                         0.753846  0.840861                    0.001994   
Decision Tree               0.728205  0.749789                    0.004987   
Random Forest                    0.8  0.864816                    0.727075   
Gradient Boosting                0.8  0.861439                    0.513596   
XGBoost                     0.764103  0.853525                      0.1127   
LightGBM                    0.789744  0.866083                    0.057846   
CatBoost                    0.753846  0.843288                    0.245005   
MLP                         0.764103  0.838863                    3.545068   
DNN                         0.774359  0.836116                    2.187616   
DCN                         0.723077  0.778793                    5.800046   
Wide_and_Deep               0.774359  0.835693                  

In [21]:
result.to_csv("result_white", index=True)